In [1]:
import pandas as pd
from google.colab import drive 

In [2]:
DATA_PATH = "gdrive/MyDrive/DataMining_project/data/"
PATH_DS_1 = DATA_PATH + "data_tpm_geo/GSE157846_tpm_matrix_all_samples_v2.csv"
PATH_DS_2 = DATA_PATH + "data_tpm_geo/GSE126797_TPM_matrix_all_datasets.csv"
PATH_DS_3 = DATA_PATH + "data_tpm_geo/GSM4838073_TPM_matrix_Breast_n3.csv"
PATH_SNODB = DATA_PATH + "data_snodb/snoDB_All_V2.0.tsv"
PATH_RIBO = DATA_PATH + "data_db_annotation/ensembl_ribosomal_proteins.tsv"

COL_NAMES = [
    "gene_id",           "gene_name",         "gene_biotype",
    "Testis_1",          "Testis_2",          "Testis_3", 
    "Skeletal_muscle_1", "Skeletal_muscle_2", "Skeletal_muscle_3",
    "Liver_1",           "Liver_2",           "Liver_3",
    "Brain_1",           "Brain_2",           "Brain_3", 
    "Breast_1",          "Breast_2",          "Breast_3",
    "Prostate_1",        "Prostate_2",        "Prostate_3",
    "Ovary_1",           "Ovary_2",           "Ovary_3"
]

# Preliminary steps


Allow the script to access the drive in order to load the files. It might require some time. It is assumed that you added the "DataMining_poject" folder as a shortcut (simply right click on the folder > "Add shortcut to drive").

In [3]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


**Load data sets** which are stored with the respective GSE name. Notice that:
- **ds1** contains expression levels for testis, skeletal muscle, liver and brain
  (3 samples each accordingly names). Moreover it contains gene names and 
  gene biotypes.
- **ds2** contains expression levels for breast (BRN117 and BRN491), prostate 
  (NPR026, NPR029, NPR036), ovary (OVN218, OVN377, OVN85) and some cell-line
  samples that are unrelated with the experiment of interest
- **ds3** contains expression levels for one breast sample

The numbering of the samples does NOT match that of the authors (it should not matter, eventually retrieve it if batch effects are found and further investigations are required)


In [4]:
ds1 = pd.read_csv(PATH_DS_1)
ds2 = pd.read_csv(PATH_DS_2)
ds3 = pd.read_csv(PATH_DS_3)

Load data from snoDB

In [5]:
snodb = pd.read_csv(PATH_SNODB, sep="\t")

# snoRNA: Merge and filter

**Merge datasets**, remove not-needed columns and rename rows and columns accordingly

In [6]:
ds_merged = ds1.merge(ds3, on="gene_id", how="outer")
ds_merged = ds_merged.merge(ds2, on="gene_id", how="outer")
ds_merged.drop(list(ds_merged.filter(regex = 'SKOV')), axis = 1, inplace = True)
ds_merged.columns = COL_NAMES
ds_merged.set_index("gene_id", inplace = True)

ds_merged.to_csv('tpm_matrix.csv')

**Create two subsets**, one containing gene information, one containing the transcript expression levels.

In [7]:
ds_info = ds_merged.iloc[:, :2]
ds_exp = ds_merged.iloc[:, 2:]

**Create a function** that, given a set of parameters, filters the dataset and returns: 
- the filtered dataset
- a dictionary containing
    - number of genes
    - number of snoRNA
    - number of host genes 
    - number of pairings of snoRNA-host genes
    - number of host-less snoRNA 
    - number of snoRNA and host-genes with missing values

In [8]:
def filter_ds(
    ds_to_filter: pd.DataFrame,
    db_snorna: pd.DataFrame,
    min_exp: float,
    min_samples: float
):
    
    """
    Filter a dataset and return some useful information.

    Given a dataframe representing an expression matrix, filter it according to
    some given parameters, then return the filtered dataframe and some 
    useful information (such as number of snoRNAs etc...). 

    Parameters
    ----------
    ds_to_filter : pd.DataFrame
        Pandas dataframe with ENSEMBL genes as rows and samples as columns. 
    db_snorna : pd.DataFrame
        Pandas dataframe containing snoDB information, with genes as rows and
        gene attributes as columns.
    min_exp : float
        Float representing the number of TPM to consider a gene expressed.
    min_samples : float
        Float representing the fraction of samples that must have the gene
        expressed in order for the gene to be retained.

    Returns
    -------
    ds_filtered : pd.DataFrame
        Pandas dataframe corresponding to the starting dataframe filtered using
        the provided parameters.
    filt_info : dict
        Dictionary containing information about the filtering process.
    """
    
    # Compute how many samples have gene expression geq than the threshold for 
    # each gene, than if it is geq than the minimal required number of samples
    # keep it, else discard it
    req_samples = ds_to_filter.shape[1] * min_samples // 1
    gene_counts = (ds_to_filter >= min_exp).sum(axis = 1)
    indexer = gene_counts >= req_samples
    ds_filtered = ds_to_filter[indexer]
    
    # Initialize counters to increment
    filt_info = {
        "num_genes": 0,
        "num_snorna": 0,
        "num_hosts": 0,
        "num_host_less": 0,
        "num_paired": 0,
        "num_with_nans": 0,
    }

    genes_filt = ds_filtered.index.values  # List of genes in filtered list
    snorna_list = list(db_snorna["ensembl_id"])  # List of snorna in db
    host_list = list(db_snorna["host_gene_id"])  # List of hosts in db
    
    filt_info["num_genes"] = len(genes_filt)  # Final number of genes kept

    for gene in genes_filt:
        
        significant = False  # Is either snorna gene or host
        
        # If gene is a snorna
        if gene in snorna_list:
            significant = True
            filt_info["num_snorna"] += 1
            
            # Retrieve host gene
            pos = db_snorna["ensembl_id"]  
            host_gene = db_snorna.loc[pos == gene, "host_gene_id"].item()

            if host_gene in genes_filt:  # Host gene is in filtered genes
                filt_info["num_paired"] += 1

            if type(host_gene) == float:  # If host gene type is NaN 
                filt_info["num_host_less"] += 1

        # If gene is a host gene
        if gene in host_list:
            significant = True
            filt_info["num_hosts"] += 1
        
        # If gene is either a snorna or a host gene
        if significant and ds_filtered.loc[[gene]].isnull().values.any():
            filt_info["num_with_nans"] += 1      

    return ds_filtered, filt_info


**Iterate through some pairs of parameters** (min_exp, min_samples) and print in a tabular way the information.

In [9]:
PARAMS = (
    (0, 0.0),
    (1, 0.1),
    (1, 0.2),
    (1, 0.3),
    (2, 0.2),
    (2, 0.5),
)

infos = list()

for params in PARAMS:
    _, info = filter_ds(ds_exp, snodb, *params)
    infos.append(info)

info_ds = pd.DataFrame.from_dict(infos)
info_ds.index = [str(par) for par in PARAMS]

info_ds

,num_genes,num_snorna,num_hosts,num_host_less,num_paired,num_with_nans
"(0, 0.0)",58884,951,1045,371,571,1
"(1, 0.1)",19341,420,913,37,363,0
"(1, 0.2)",13509,396,800,29,339,0
"(1, 0.3)",11690,385,738,25,329,0
"(2, 0.2)",9492,378,622,20,320,0
"(2, 0.5)",5145,345,343,11,252,0


# snoRNA: Export

**Rerun** using final parameters

In [10]:
final_params = (1, 0.2)
filt_mat, filt_info = filter_ds(ds_exp, snodb, *final_params)
filt_mat.dropna(inplace = True)
filt_mat = filt_mat.round(2)

/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


**Save** expression matrix data to csv

In [11]:
filt_mat.to_csv(DATA_PATH + "data_expansion_input/filtered_dataset.csv")

Retrieve **list of snoRNA to expand**

In [12]:
snorna_list = [
    snorna 
    for snorna in filt_mat.index 
    if snorna in list(snodb["ensembl_id"])
]

Save list of snoRNA **in plain txt format**

In [13]:
with open(DATA_PATH + "data_expansion_input/exp_snorna.txt", "w") as txt_file:
    for snorna in snorna_list:
        txt_file.write(f"{snorna}\n")

Save list of **snoRNA in lgn format**

In [14]:
with open(DATA_PATH + "data_expansion_input/snorna_net.lgn", "w") as lgn_file:
    lgn_file.write("from,to\n")
    for snorna in snorna_list:
        lgn_file.write(f"{snorna},{snorna}\n")

# ribosomal proteins: Export

Read list of all ribosomal proteins according to HUGO

In [15]:
all_ribo = pd.read_csv(PATH_RIBO, sep="\t")

Filter ensemble ids of non-mitochondrial ribo-proteins

In [16]:
non_mito_ribo = all_ribo[~all_ribo['Group name'].str.contains(
    "mitochondrial", case=False)]

Filter out only ribosomal proteins present in the filtered dataset

In [17]:
ribo_list = [
    ribo_p
    for ribo_p in list(non_mito_ribo["Ensembl gene ID"])
    if ribo_p in filt_mat.index 
]

Save list of ribosomal proteins to expand in plain txt format

In [18]:
with open(DATA_PATH + "data_expansion_input/ribosomal_list.txt", "w") as txt:
    for ribo in ribo_list:
        txt.write(f"{ribo}\n")

Save list of **ribosomal proteins in lgn format**

In [19]:
with open(DATA_PATH + "data_expansion_input/ribosomal_net.lgn", "w") as lgn:
    lgn.write("from,to\n")
    for ribo_p in ribo_list:
        lgn.write(f"{ribo_p},{ribo_p}\n")

Save list of **ribosomal and snoRNA in a single lgn** file

In [20]:
snorna_ribo_list = snorna_list + ribo_list
with open(DATA_PATH + "data_expansion_input/snorna_ribo_net.lgn", "w") as lgn:
    lgn.write("from,to\n")
    for sno_ribo in snorna_ribo_list:
        lgn.write(f"{sno_ribo},{sno_ribo}\n")